In [14]:
import cv2     # for capturing videos
import math   # for mathematical operations
import matplotlib.pyplot as plt    # for plotting the images
import numpy as np    # for mathematical operations
import pandas as pd
from keras.preprocessing import image   # for preprocessing the images
from keras.utils import np_utils
from skimage.transform import resize   # for resizing images
from sklearn.model_selection import train_test_split
from glob import glob
from tqdm import tqdm
import os

import os
import cv2
import math
import random
import numpy as np
import datetime as dt
import tensorflow as tf
from moviepy.editor import *
from collections import deque
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from keras.layers import Conv2D
from keras.layers import Input
from keras.layers import MaxPooling2D, UpSampling2D
from keras.models import Model
from keras.datasets.mnist import load_data
from numpy import reshape
import matplotlib.pyplot as plt
import os
import glob
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import cv2
from keras.preprocessing import image
from keras.models import *
from tensorflow.keras.optimizers import Adam, RMSprop
from keras.callbacks import EarlyStopping
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv2D
from keras.layers import MaxPooling2D, Dropout, UpSampling2D
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model

In [5]:
import tensorflow as tf

def hard_sigmoid(x):
    return tf.nn.relu6(x + 3) / 6

def hard_swish(x):
    return x * hard_sigmoid(x)

def conv_bn_activation(x, filters, kernel_size, strides=1, padding='same', activation=None):
    x = Conv2D(filters=filters, kernel_size=kernel_size, strides=strides, padding=padding)(x)
    x = BatchNormalization()(x)
    if activation:
        x = Activation(hard_swish)(x)
    return x

def bottleneck(x, filters, kernel_size, strides=1, expansion=4, activation=None):
    channel_axis = 1 if tf.keras.backend.image_data_format() == 'channels_first' else -1
    input_shape = tf.keras.backend.int_shape(x)
    input_filters = input_shape[channel_axis]
    t = Conv2D(input_filters * expansion, 1, padding='same')(x)
    t = BatchNormalization()(t)
    if activation == 'hard_sigmoid':
        t = Activation(hard_sigmoid)(t)
    else:
        t = Activation(hard_swish)(t)
    t = conv_bn_activation(t, filters, kernel_size, strides)
    t = BatchNormalization()(t)
    if input_filters == filters and strides == 1:
        shortcut = x
    else:
        shortcut = conv_bn_activation(x, filters, 1, strides)
    output = add([shortcut, t])
    return output

def MobileNetV3(input_shape=(224, 224, 3), alpha=1.0, classes=5):
    img_input = Input(shape=input_shape)

    # Stem
    x = conv_bn_activation(img_input, int(16 * alpha), 3, strides=2, activation='hard_swish')

    # Blocks
    x = bottleneck(x, int(16 * alpha), 3, expansion=1, activation='hard_sigmoid')
    x = bottleneck(x, int(24 * alpha), 3, strides=2, expansion=4, activation='hard_sigmoid')
    x = bottleneck(x, int(24 * alpha), 3, expansion=3, activation='hard_sigmoid')
    x = bottleneck(x, int(40 * alpha), 5, strides=2, expansion=3, activation='hard_sigmoid')
    x = bottleneck(x, int(40 * alpha), 5, expansion=3, activation='hard_sigmoid')
    x = bottleneck(x, int(40 * alpha), 5, expansion=3, activation='hard_sigmoid')
    x = bottleneck(x, int(80 * alpha), 3, expansion=6, activation='hard_swish')
    x = bottleneck(x, int(80 * alpha), 3, expansion=6, activation='hard_swish')
    x = bottleneck(x, int(80 * alpha), 3, expansion=6, activation='hard_swish')
    x = bottleneck(x, int(80 * alpha), 3, expansion=6, activation='hard_swish')
    x = bottleneck(x, int(112 * alpha), 5, expansion=6, activation='hard_swish')
    x = bottleneck(x, int(112 * alpha), 5, expansion=6, activation='hard_swish')
    x = bottleneck(x, int(160 * alpha), 5, expansion=6, activation='hard_swish')
    x = bottleneck(x, int(160 * alpha), 5, expansion=6, activation='hard_swish')
    x = bottleneck(x, int(160 * alpha), 5, expansion=6, activation='hard_swish')

    # Head
    x = conv_bn_activation(x, int(960 * alpha),1, padding='same', activation='hard_swish')
    x = GlobalAveragePooling2D()(x)
    x = Reshape((1, 1, int(960 * alpha)))(x)
    x = Conv2D(int(1280 * alpha), 1, padding='same')(x)
    x = Activation(hard_swish)(x)
    x = Conv1D(classes, 1, padding='same', activation='softmax')(x)
    x = Reshape((classes,))(x)
    
    model = tf.keras.models.Model(img_input, x, name='mobilenetv3')

    return model

In [6]:
from keras.models import *

In [7]:
model = MobileNetV3()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
loss = tf.keras.losses.CategoricalCrossentropy()
metrics = [tf.keras.metrics.CategoricalAccuracy()]
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [8]:
model.summary()

Model: "mobilenetv3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_38 (Conv2D)             (None, 112, 112, 16  448         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_52 (BatchN  (None, 112, 112, 16  64         ['conv2d_38[0][0]']              
 ormalization)                  )                                                       

                                                                                                  
 activation_22 (Activation)     (None, 56, 56, 72)   0           ['batch_normalization_63[0][0]'] 
                                                                                                  
 conv2d_47 (Conv2D)             (None, 28, 28, 40)   72040       ['activation_22[0][0]']          
                                                                                                  
 conv2d_48 (Conv2D)             (None, 28, 28, 40)   1000        ['add_17[0][0]']                 
                                                                                                  
 batch_normalization_64 (BatchN  (None, 28, 28, 40)  160         ['conv2d_47[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 batch_nor

 activation_26 (Activation)     (None, 28, 28, 480)  0           ['batch_normalization_77[0][0]'] 
                                                                                                  
 conv2d_57 (Conv2D)             (None, 28, 28, 80)   345680      ['activation_26[0][0]']          
                                                                                                  
 batch_normalization_78 (BatchN  (None, 28, 28, 80)  320         ['conv2d_57[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 batch_normalization_79 (BatchN  (None, 28, 28, 80)  320         ['batch_normalization_78[0][0]'] 
 ormalization)                                                                                    
                                                                                                  
 add_22 (A

                                                                                                  
 batch_normalization_92 (BatchN  (None, 28, 28, 112)  448        ['batch_normalization_91[0][0]'] 
 ormalization)                                                                                    
                                                                                                  
 add_26 (Add)                   (None, 28, 28, 112)  0           ['add_25[0][0]',                 
                                                                  'batch_normalization_92[0][0]'] 
                                                                                                  
 conv2d_67 (Conv2D)             (None, 28, 28, 672)  75936       ['add_26[0][0]']                 
                                                                                                  
 batch_normalization_93 (BatchN  (None, 28, 28, 672)  2688       ['conv2d_67[0][0]']              
 ormalizat

 conv1d_1 (Conv1D)              (None, 1, 1, 5)      6405        ['activation_35[0][0]']          
                                                                                                  
 reshape_3 (Reshape)            (None, 5)            0           ['conv1d_1[0][0]']               
                                                                                                  
Total params: 17,251,357
Trainable params: 17,231,965
Non-trainable params: 19,392
__________________________________________________________________________________________________


In [33]:
def preprocess_video(video_path, input_shape, num_of_frame_per_video):
    # Load video and get properties
    cap = cv2.VideoCapture(video_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))

    # Initialize array to store frames
    all_frames = np.zeros((frame_count, input_shape[0], input_shape[1], input_shape[2]))
    frames = np.zeros((num_of_frame_per_video, input_shape[0], input_shape[1], input_shape[2]))
    
    # Read frames and resize
    for i in range(frame_count):
        ret, frame = cap.read()
        frame_resized = cv2.resize(frame, (input_shape[0], input_shape[1]))
        all_frames[i] = frame_resized
    
    # extract equal distance frames from all_frames
    j = 0
    distance_frame = frame_count//num_of_frame_per_video
    for i in range(num_of_frame_per_video):
        frames[i] = all_frames[j]
        j = j + distance_frame

    
    # Normalize pixel values to [0, 1]
    frames = frames / 255.0
    
    # Close video capture
    cap.release()
    
    return frames

In [17]:
def create_dataset(input_shape, num_of_frame_per_video):
    import random
    # Declaring Empty Lists to store the features and labels values.
    features = []
    labels = []
    # Iterating through all the classes mentioned in the classes list
    for class_index, class_name in enumerate(classes_list):
        print(f'Extracting Data of Class: {class_name}')
        
        # Getting the list of video files present in the specific class name directory
        files_list = os.listdir(os.path.join(dataset_directory, class_name))
        print(files_list)

        # Iterating through all the files present in the files list
        for file_name in files_list:
            
            if file_name == '.DS_Store':
                continue
            # Construct the complete video path
            video_file_path = os.path.join(dataset_directory, class_name, file_name)
            print(video_file_path)

            # Calling the frame_extraction method for every video file path
            frames = preprocess_video(video_file_path, input_shape, num_of_frame_per_video)
          

            # Appending the frames to a temporary list.
#             features.append(frames)
            features += list(frames)
        
            labels.extend([class_index] * num_of_frame_per_video)
            
    # Converting the features and labels lists to numpy arrays
    features = np.array(features)
    labels = np.array(labels) 
    
    return features, labels  

In [41]:
# dataset_directory = '/Users/parthkalathia/Desktop/NEW_DATA'
dataset_directory = '/Users/parthkalathia/Desktop/CNN_SCRATCH/FINALDATASET_V2'
classes_list = ['TALKING', 'HEADDOWN', 'YAWNING', 'MOBILE', 'NORMAL']


In [42]:
features, labels = create_dataset((224,224,3), 10)

Extracting Data of Class: TALKING
['T14.jpg', '.DS_Store', 'T8.MOV', 'T9.MOV', 'T1.MOV', 'T2.MOV', 'T3.MOV', 'T10.MOV', 'T7.MOV', 'T6.MOV', 'T5.MOV', 'T12.mp4', 'T11.mp4']
/Users/parthkalathia/Desktop/CNN_SCRATCH/FINALDATASET_V2/TALKING/T14.jpg
/Users/parthkalathia/Desktop/CNN_SCRATCH/FINALDATASET_V2/TALKING/T8.MOV
/Users/parthkalathia/Desktop/CNN_SCRATCH/FINALDATASET_V2/TALKING/T9.MOV
/Users/parthkalathia/Desktop/CNN_SCRATCH/FINALDATASET_V2/TALKING/T1.MOV
/Users/parthkalathia/Desktop/CNN_SCRATCH/FINALDATASET_V2/TALKING/T2.MOV
/Users/parthkalathia/Desktop/CNN_SCRATCH/FINALDATASET_V2/TALKING/T3.MOV
/Users/parthkalathia/Desktop/CNN_SCRATCH/FINALDATASET_V2/TALKING/T10.MOV
/Users/parthkalathia/Desktop/CNN_SCRATCH/FINALDATASET_V2/TALKING/T7.MOV
/Users/parthkalathia/Desktop/CNN_SCRATCH/FINALDATASET_V2/TALKING/T6.MOV
/Users/parthkalathia/Desktop/CNN_SCRATCH/FINALDATASET_V2/TALKING/T5.MOV
/Users/parthkalathia/Desktop/CNN_SCRATCH/FINALDATASET_V2/TALKING/T12.mp4
/Users/parthkalathia/Desktop/CNN_

/Users/parthkalathia/Desktop/CNN_SCRATCH/FINALDATASET_V2/NORMAL/N4.mp4
/Users/parthkalathia/Desktop/CNN_SCRATCH/FINALDATASET_V2/NORMAL/N6.mp4
/Users/parthkalathia/Desktop/CNN_SCRATCH/FINALDATASET_V2/NORMAL/N7.mp4
/Users/parthkalathia/Desktop/CNN_SCRATCH/FINALDATASET_V2/NORMAL/N3.mp4
/Users/parthkalathia/Desktop/CNN_SCRATCH/FINALDATASET_V2/NORMAL/N2.mp4
/Users/parthkalathia/Desktop/CNN_SCRATCH/FINALDATASET_V2/NORMAL/N1.mp4
/Users/parthkalathia/Desktop/CNN_SCRATCH/FINALDATASET_V2/NORMAL/N23.mp4
/Users/parthkalathia/Desktop/CNN_SCRATCH/FINALDATASET_V2/NORMAL/N22.mp4
/Users/parthkalathia/Desktop/CNN_SCRATCH/FINALDATASET_V2/NORMAL/N20.mp4
/Users/parthkalathia/Desktop/CNN_SCRATCH/FINALDATASET_V2/NORMAL/N21.mp4
/Users/parthkalathia/Desktop/CNN_SCRATCH/FINALDATASET_V2/NORMAL/N19.mp4
/Users/parthkalathia/Desktop/CNN_SCRATCH/FINALDATASET_V2/NORMAL/N18.mp4
/Users/parthkalathia/Desktop/CNN_SCRATCH/FINALDATASET_V2/NORMAL/N24.mp4
/Users/parthkalathia/Desktop/CNN_SCRATCH/FINALDATASET_V2/NORMAL/N17.mp

In [43]:
one_hot_encoded_labels = to_categorical(labels)

In [44]:
from sklearn.utils import shuffle
X_train, Y_train = shuffle(features, one_hot_encoded_labels, random_state=23)

In [48]:
X_train.shape

(1150, 224, 224, 3)

In [45]:
early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 15, mode = 'min', restore_best_weights = True)

# Adding loss, optimizer and metrics values to the model.
model.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ["accuracy"])

In [49]:
model_training_history = model.fit(X_train, Y_train , shuffle = True, epochs = 5 , batch_size= 10, validation_split=0.1, callbacks = [early_stopping_callback])

Epoch 1/5
104/104 [==============================] - 951s 9s/step - loss: 1.5707 - accuracy: 0.3894 - val_loss: 10.8608 - val_accuracy: 0.1130
Epoch 2/5
104/104 [==============================] - 905s 9s/step - loss: 1.3069 - accuracy: 0.4213 - val_loss: 7.5114 - val_accuracy: 0.1304
Epoch 3/5
104/104 [==============================] - 884s 9s/step - loss: 1.2016 - accuracy: 0.5024 - val_loss: 13.9904 - val_accuracy: 0.1304
Epoch 4/5
104/104 [==============================] - 880s 8s/step - loss: 1.1121 - accuracy: 0.5343 - val_loss: 9.9988 - val_accuracy: 0.1304
Epoch 5/5
104/104 [==============================] - 947s 9s/step - loss: 1.0772 - accuracy: 0.5546 - val_loss: 12.5197 - val_accuracy: 0.1304
